### Autogen uses async functionality for real time agent interactions
- If Agent calls an API , we can keep it in background and carry on with other tasks while it waits for response
- Synchronous -1 by 1 sequential execution
- async - co routine execution (multiple task execution simultaneously)

In [1]:
# Sync process
import time
def brew_coffee():
    print("Brewing Coffee")
    time.sleep(3) # 3 Minutes
    print("Coffee Ready")

def toast_bread():
    print("Toasting Bread")
    time.sleep(2) # 2 Minutes
    print("bread Ready")


start = time.time()

coffee = brew_coffee()
# time.sleep(2)
bread = toast_bread()

end = time.time()


print(f"Time : {end - start:.2f} minutes")


Brewing Coffee
Coffee Ready
Toasting Bread
bread Ready
Time : 5.01 minutes


In [2]:
# Async process

import asyncio
import time

async def brew_coffee():
    print("Starting Brewing Coffee")
    await asyncio.sleep(3)
    print("Coffee Ready")

async def toast_bread():
    print("Start Toasting bread")
    await asyncio.sleep(2)
    print("Bread Ready")


start = time.time()

coffee = brew_coffee()
bread = toast_bread()

results = await asyncio.gather(coffee,bread)

end = time.time()

print(f"Time : {end - start:.2f} minutes")




Starting Brewing Coffee
Start Toasting bread
Bread Ready
Coffee Ready
Time : 3.00 minutes


In [5]:
import asyncio
import time

async def brew_coffee_async():
    print("Starting Brewing Coffee")
    await asyncio.sleep(3)
    print("Coffee Ready")

async def toast_bread_async():
    print("Start Toasting bread")
    await asyncio.sleep(2)
    print("Bread Ready")

async def main_individual():
    start = time.time()
    coffee_task = asyncio.create_task(brew_coffee_async())
    bread_task = asyncio.create_task(toast_bread_async())

    coffee = await coffee_task
    bagel = await bread_task

    end = time.time()
    print(f"Time : {end - start:.2f} minutes")


await (main_individual())


Starting Brewing Coffee
Start Toasting bread
Bread Ready
Coffee Ready
Time : 3.01 minutes


# Now more on AgentChat ( What we already saw)
- Agentchat gives us a async powered way to handle agent conversation
- It helps us to define agents and enable them to chat , collaborate and solve tasks
- event driven i.e response to task as they come
- features are - customization with prompts we can do , we can connect it to LLMs 
- This is a high level API makes development is very easy 

# Customizing the Agents and prompt engineering 

In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4', api_key=api_key)

### Agent Customization

#- We can assign a role to our agent
#- help in fitting agent to specific use case
# This will help us in multi agent frameworks later

asssistant = AssistantAgent(
    name = 'politicalscience_expert',
    model_client=model_client,
    description='A knowledgeable assistant with expertise in political science',
    system_message='You are a political science expert with deep knowledge of world politics. Provide detailed and accuragte answers about political events,figures and timelines and explanation'
)

async def test_politicalscience_expert():
    result = await asssistant.run(task = 'What is the future of India and US relationship?')
    print(result.messages[-1].content)

await test_politicalscience_expert()

Predicting the future of international relations involves a good deal of uncertainty and speculation. However, a few trends hint at the possible trajectory of the relationship between India and the United States. 

1. Shared Democratic Values: As two of the world's largest democracies, the US and India share many values, such as the importance of a democratic government and the rule of law. These shared values remain a cornerstone in their partnership.

2. Strategic Partner: India's location makes it a key strategic partner for the US, especially in relation to China. US regards India as an important partner in the Indo-Pacific region. If tensions between the US and China continue to escalate, the strategic significance of India will likely increase.

3. Trade Relations: Economic and trade relations between the two countries have been a friction point. Both sides have expressed interest in a trade deal, but disagreements over specific terms have hampered progress. The future relationsh

# Using tools for MS Autogen

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4', api_key=api_key)

def get_weather(city:str) -> str:
    """
    Fetches the weather for a given city.
    This is a placeholder function and should be replaced with actual weather fetching logic.
    For example, you could use a weather API to get real-time data.
    :param city: Name of the city to fetch weather for.
    :return: Weather information as a string.
    """
    # Placeholder function to simulate weather fetching
    return f"The weather in {city} is cloudy with a high of -5 degree celsius."

assistant = AssistantAgent(
    name = 'weather_assistant',
    model_client=model_client,
    system_message='You are a weather assistant, Use the right tool when asked about the weather in a city.',
    tools= [get_weather]
)

async def test_weather_tool():
    result = await assistant.run(task = 'What is the weather in Toronto?')
    print(result.messages[-1].content)

await test_weather_tool()

The weather in Toronto is cloudy with a high of -5 degree celsius.


# Messages in Autogen
-  We can imagine messages as the way agent communicate each other - chatting with our Friend. 

- When we communicate with the agents -----> sending a message
- when it responds ---> it too sends a message

- TextMessage 
- ImageMessage
- ToolMessage

In [5]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

# Assistant Agent
assistant_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message='You are a helpful assistant. Answer questions accurately and provide detailed information.'
)

# User Proxy Agent
user_proxy_agent = UserProxyAgent(
    name="user_proxy"
)
async def test_direct_interaction():
    print("\n--- Direct Interaction Test ---")
    
    # User proxy sends message to assistant
    text_msg = TextMessage(content='Who is the prime minister of India? Please provide current information.', source='user_proxy')
    
    # Assistant processes the message
    result = await assistant_agent.run(task=text_msg)
    
    print("Assistant's response:")
    print(result.messages[-1].content)

await test_direct_interaction()


--- Direct Interaction Test ---
Assistant's response:
As of October 2023, the Prime Minister of India is Narendra Modi. He has been in office since May 26, 2014, after leading the Bharatiya Janata Party (BJP) to victory in the general elections.


In [8]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

# Assistant Agent
assistant_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message='You are a helpful assistant. Answer questions accurately and provide detailed information.'
)

# User Proxy Agent
user_proxy_agent = UserProxyAgent(
    name="user_proxy"
)

async def interactive_chat():
    """Interactive chat function that allows users to input questions"""
    print("=== Welcome to Interactive AutoGen Chat ===")
    print("Type 'quit', 'exit', or 'bye' to end the conversation")
    print("=" * 50)
    
    while True:
        try:
            # Get user input
            user_question = input("\nYou: ").strip()
            
            # Check for exit commands
            if user_question.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\nGoodbye! Thanks for chatting!")
                break
            
            # Skip empty inputs
            if not user_question:
                print("Please enter a question or type 'quit' to exit.")
                continue
            
            print("\nThinking...")
            
            # Create message and get response
            text_msg = TextMessage(content=user_question, source='user_proxy')
            result = await assistant_agent.run(task=text_msg)
            
            # Display assistant's response
            print(f"\nAssistant: {result.messages[-1].content}")
            print("-" * 50)
            
        except KeyboardInterrupt:
            print("\n\nChat interrupted by user. Goodbye!")
            break
        except Exception as e:
            print(f"\nError occurred: {e}")
            print("Please try again or type 'quit' to exit.")

async def single_question_mode():
    """Single question mode - ask one question and get answer"""
    user_question = input("Enter your question: ").strip()
    
    if not user_question:
        print("No question provided.")
        return
    
    print("\nProcessing your question...")
    
    try:
        text_msg = TextMessage(content=user_question, source='user_proxy')
        result = await assistant_agent.run(task=text_msg)
        
        print(f"\nAnswer: {result.messages[-1].content}")
        
    except Exception as e:
        print(f"Error: {e}")

async def main():
    """Main function to choose interaction mode"""
    print("Choose interaction mode:")
    print("1. Interactive chat (continuous conversation)")
    print("2. Single question mode")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        
        if choice == '1':
            await interactive_chat()
            break
        elif choice == '2':
            await single_question_mode()
            break
        else:
            print("Invalid choice. Please enter 1 or 2.")

# Run the main function
await main()

Choose interaction mode:
1. Interactive chat (continuous conversation)
2. Single question mode
=== Welcome to Interactive AutoGen Chat ===
Type 'quit', 'exit', or 'bye' to end the conversation

Thinking...

Assistant: AI in retail enhances customer experiences through personalized recommendations and efficient customer service while optimizing operations by streamlining inventory management and demand forecasting. By analyzing vast amounts of data, AI enables retailers to offer targeted marketing strategies and improve supply chain efficiency.
--------------------------------------------------

Thinking...

Assistant: The last question you asked was about the role of AI in retail, specifically requesting a two-sentence explanation.
--------------------------------------------------

Goodbye! Thanks for chatting!


In [9]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage,MultiModalMessage
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")
model_client = OpenAIChatCompletionClient(model='gpt-4o',api_key=api_key)

# Simple text message 
agent = AssistantAgent(
    name = "text_agent",
    model_client=model_client,
    system_message='You are a helpful assistant. answer question accurately'
)

async def test_text_messages():
    text_msg = TextMessage(content = 'Who is prime minister of India?', source='user')
    result = await agent.run(task=text_msg)
    print(result.messages[-1].content)

await test_text_messages()

As of October 2023, the Prime Minister of India is Narendra Modi. He has been in office since May 26, 2014.


In [10]:
async def test_multi_modal():

    response = requests.get('https://picsum.photos/id/21/200/300') # 23 for the image of folks
    pil_image = Image.open(BytesIO(response.content))
    ag_image = AGImage(pil_image)

    multi_modal_msg = MultiModalMessage(
        content = ['What is in the image?',ag_image],
        source='user'
    )

    result = await agent.run(task=multi_modal_msg)
    print(result.messages[-1].content)


await test_multi_modal()

The image shows a pair of white high-heeled shoes placed against a red, textured background with white splatters.


# Mention about on_message() and on_messages_stream()

# Structured output via autogen

In [12]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from pydantic import BaseModel
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

from pydantic import BaseModel

class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles: int

model_client = OpenAIChatCompletionClient(
    model = 'gpt-4o',
    api_key=api_key,
    response_format=PlanetInfo
    )

unstructured_model_client = OpenAIChatCompletionClient(
    model = 'gpt-4o',
    api_key=api_key,
    # response_format=PlanetInfo
    )

agent = AssistantAgent(
    name='planet_agent',
    model_client=unstructured_model_client,
    system_message="You are a helpful assistant that provides information about planets." 
)
async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.",source='User')
    result = await agent.run(task=task)
    structured_response = result.messages[-1].content
    print(structured_response)

await test_structured_output()

Mars, often referred to as the "Red Planet," is the fourth planet from the Sun in our solar system. It has been a point of interest for scientists and astronomers for centuries due to its similarities and differences with Earth. Here are some key details about Mars:

1. **Basic Characteristics:**
   - **Diameter:** Approximately 6,779 kilometers (4,212 miles), making it about half the diameter of Earth.
   - **Mass:** About 0.11 times that of Earth.
   - **Surface Gravity:** Approximately 0.38 of Earth's gravity.
   - **Length of Day:** A Martian day, or sol, is approximately 24.6 hours.
   - **Length of Year:** About 687 Earth days.

2. **Orbit and Rotation:**
   - Mars has an elliptical orbit, leading to variation in its distance from the Sun, ranging from about 206 million km (128 million miles) to 249 million km (155 million miles).

3. **Atmosphere:**
   - The atmosphere is thin, composed mostly of carbon dioxide (95.3%), with traces of nitrogen, argon, oxygen, and water vapor.
  

In [13]:
agent = AssistantAgent(
    name='planet_agent',
    model_client=model_client,
    system_message="You are a helpful assistant that provides information about planets."
)
async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.",source='User')
    result = await agent.run(task=task)
    structured_response = result.messages[-1].content
    print(structured_response)

await test_structured_output()

{"name":"Mars","color":"Red","distance_miles":141600000}
